In [23]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os

In [24]:
def directory(files, argumento):
    directory = glob.glob(os.path.join(files, argumento))
    lista = []
    for arquivo in directory:
        x = pd.read_csv(arquivo, header= None)
        lista.append(x)
    df_temp = pd.concat(lista, axis= 0, ignore_index= True)
    return df_temp

df_sug = directory(pasta.p_82, 'DEP*.txt')
df_mov = directory(pasta.p_82, 'MOV*.txt')
df_prod = pd.read_excel(ar_xlsx.ar_96, usecols=['CODPROD','DESCRICAO', 'QTUNITCX', 'QTTOTPAL', 'OBS2','RUA', 'PREDIO', 'APTO'])
df_acesso = pd.read_excel(ar_xlsx.ar_60, usecols= ['CODPROD','QTOS', 'QT'])


c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:

df_sug.columns = col_name.c82
df_mov.columns = col_name.cMovi

df_prod = df_prod.loc[df_prod['RUA'].between(1,39)]
df_prod['OBS2'] = df_prod['OBS2'].fillna("Ativos")
df_prod["PRODUTO"] = df_prod['CODPROD'].astype(str)

x = df_acesso.groupby('CODPROD').agg(
ACESSO = ("QTOS", "sum")
).reset_index()

df = df_sug.merge(df_prod, left_on= 'COD', right_on= 'CODPROD', how= 'left')
df = df.merge(x, left_on='COD', right_on='CODPROD', how= 'left')
df = df.merge(df_mov, left_on='COD', right_on='COD', how="left")

df[['COM FATOR', 'QTTOTPAL', 'CAP']] = df[['COM FATOR', 'QTTOTPAL','CAP']].fillna(0)
df['PL_SUG'] = df['COM FATOR'].astype(int) / df['QTTOTPAL'].astype(int)
df['PL_CAP'] = df['CAP'].astype(int) / df['QTTOTPAL'].astype(int)
df['CONTAGEM'] = 
drop_col = []

In [30]:
display(df.columns)

Index(['COD', 'DESCRIÇÃO', 'EMBALAGEM_x', 'UNID.', 'DEP', 'RUA_x', 'PREDIO_x',
       'NIVEL', 'APTO_x', 'MÊS 1', 'MÊS 2', 'MÊS 3', 'TIPO', 'CAP', '1 DIA',
       'COM FATOR', 'VARIAÇÃO', '%_x', 'CODPROD_x', 'DESCRICAO', 'OBS2',
       'QTUNITCX', 'QTTOTPAL', 'RUA_y', 'PREDIO_y', 'APTO_y', 'PRODUTO',
       'CODPROD_y', 'ACESSO', 'DESC', 'EMBALAGEM_y', 'UNID', 'MOVI', '%_y',
       '%_ACUM', 'CLASSE', 'PL_SUG', 'PL_CAP'],
      dtype='object')